# Table of Contents
* [Lecture 2D - Introduction to Time Series Data](#Lecture-8---Introduction-to-Time-Series-Data)
	* &nbsp;
		* [Content](#Content)
		* [Learning Outcomes](#Learning-Outcomes)
* [Importing Time Series Data](#Importing-Time-Series-Data)
* [Converting into Time Series Data](#Converting-into-Time-Series-Data)
* [Filtering Time Series Data](#Filtering-Time-Series-Data)
* [Resampling](#Resampling)
	* &nbsp;
		* [Moving (rolling/running) statistics](#Moving-%28rolling/running%29-statistics)
	* [Shift operations](#Shift-operations)
		* [Exercise:](#Exercise:)


# Lecture 2D - Introduction to Time Series Data

---

### Content

1. Importing time series data
2. Time series data types and conversions
3. Time series filtering
4. Time series resampling
5. Plotting time series

### Learning Outcomes

At the end of this lecture, you should be able to:

* import time series data
* convert datasets into appropriate time series data types
* filter dataframes based on time series conditions
* perform resampling of time series data
* perform running averages on time series data
* visualise time series data

The overall goal of Pandas is that of becoming "the most powerful and flexible open source data analysis manipulation tool available in any language", and it is already well on its way toward realizing this. One of the domains where Pandas has been excelling and has become a proven a tool is in the domain of time series data analysis. 

Time series data is a sequence of data points that comprises of measurements made over a time interval, where the time interval is continuous, having the same distance between consecutive data points, while generating at most one data point for each given moment in time.

Time series analysis is an substantive topic. The aim here will be to provide a brief introduction on how to process, manipulate and visualise time series data using a small subset of Pandas capabilities.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from pylab import rcParams #this module gives us some controls over plot rendering attributes
rcParams['figure.figsize'] = 15, 10

In [4]:
#this line enables the plots to be embedded into the notebook
%matplotlib inline

# Importing Time Series Data

Below is a dataset extracted from Yahoo Finance showing the daily Apple stock price movements from 1980 to February 2016.

In [5]:
ts_data = pd.read_csv('appleStockPrice.csv')
ts_data.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-02-03,95.000000,96.839996,94.080002,96.349998,45964300,95.830001
1,2016-02-02,95.419998,96.040001,94.279999,94.480003,37357200,93.970098
2,2016-02-01,96.470001,96.709999,95.400002,96.430000,40943500,95.909571
3,2016-01-29,94.790001,97.339996,94.349998,97.339996,64416500,96.814656
4,2016-01-28,93.790001,94.519997,92.389999,94.089996,55678800,93.582196


In [6]:
ts_data.tail()

,Date,Open,High,Low,Close,Volume,Adj Close
8857,1980-12-18,26.625000,26.750000,26.625,26.625,18362400,0.401907
8858,1980-12-17,25.875000,25.999999,25.875,25.875,21610400,0.390586
8859,1980-12-16,25.375000,25.375000,25.250,25.250,26432000,0.381151
8860,1980-12-15,27.375001,27.375001,27.250,27.250,43971200,0.411342
8861,1980-12-12,28.750000,28.875000,28.750,28.750,117258400,0.433984


Notice that the data that we imported has the order of observations in a descending order in respect to Date. We will deal with this later.

Examine the data types for each of the columns.

In [7]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8862 entries, 0 to 8861
Data columns (total 7 columns):
Date         8862 non-null object
Open         8862 non-null float64
High         8862 non-null float64
Low          8862 non-null float64
Close        8862 non-null float64
Volume       8862 non-null int64
Adj Close    8862 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 484.7+ KB


Notice that the 'Date' column is an 'object' data type. This means that it has been interepreted as a 'string' rather than as a 'date' data type.

# Converting into Time Series Data

Below is an example of how we can convert a column that is interpreted as a string, into a datetime datatype.

In [8]:
ts_data['Date'] = pd.to_datetime(ts_data['Date'], format='%Y-%m-%d')
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8862 entries, 0 to 8861
Data columns (total 7 columns):
Date         8862 non-null datetime64[ns]
Open         8862 non-null float64
High         8862 non-null float64
Low          8862 non-null float64
Close        8862 non-null float64
Volume       8862 non-null int64
Adj Close    8862 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 484.7 KB


Notice the 'format' specification and how it fits exactly the format of the original string.

In [9]:
print ts_data.info()
ts_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8862 entries, 0 to 8861
Data columns (total 7 columns):
Date         8862 non-null datetime64[ns]
Open         8862 non-null float64
High         8862 non-null float64
Low          8862 non-null float64
Close        8862 non-null float64
Volume       8862 non-null int64
Adj Close    8862 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 484.7 KB
None


,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-02-03,95.000000,96.839996,94.080002,96.349998,45964300,95.830001
1,2016-02-02,95.419998,96.040001,94.279999,94.480003,37357200,93.970098
2,2016-02-01,96.470001,96.709999,95.400002,96.430000,40943500,95.909571
3,2016-01-29,94.790001,97.339996,94.349998,97.339996,64416500,96.814656
4,2016-01-28,93.790001,94.519997,92.389999,94.089996,55678800,93.582196


The Date column is now a datetime64 data type. Notice that the appearance of the Date column has not changed, which is why it is important to check that the data types are as you would like them to be for each column.



**Exercise:** Use the pd.read_clipboard() function to read the below data into a dataframe. Then convert the column Date into a datetime data type that has the format of Year/Month/Day as above:  

We can perform more powerful manipulation and processing if we make the Date column the index.

In [10]:
ts_data = ts_data.set_index(['Date'])
ts_data.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-02-03,95.000000,96.839996,94.080002,96.349998,45964300,95.830001
2016-02-02,95.419998,96.040001,94.279999,94.480003,37357200,93.970098
2016-02-01,96.470001,96.709999,95.400002,96.430000,40943500,95.909571
2016-01-29,94.790001,97.339996,94.349998,97.339996,64416500,96.814656
2016-01-28,93.790001,94.519997,92.389999,94.089996,55678800,93.582196


It is important to know how to manually convert columns into datetime and make them into a dataframe index; however, when reading in a csv file, we can do all of the above automatically in future by specifying a couple of parameters. 

In [12]:
ts_data = pd.read_csv('appleStockPrice.csv', index_col='Date', parse_dates=True)
print ts_data.info()
ts_data.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8862 entries, 2016-02-03 to 1980-12-12
Data columns (total 6 columns):
Open         8862 non-null float64
High         8862 non-null float64
Low          8862 non-null float64
Close        8862 non-null float64
Volume       8862 non-null int64
Adj Close    8862 non-null float64
dtypes: float64(5), int64(1)
memory usage: 484.6 KB
None


,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-02-03,95.000000,96.839996,94.080002,96.349998,45964300,95.830001
2016-02-02,95.419998,96.040001,94.279999,94.480003,37357200,93.970098
2016-02-01,96.470001,96.709999,95.400002,96.430000,40943500,95.909571
2016-01-29,94.790001,97.339996,94.349998,97.339996,64416500,96.814656
2016-01-28,93.790001,94.519997,92.389999,94.089996,55678800,93.582196


# Filtering Time Series Data

In [13]:
ts_data['2016']

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-02-03,95.000000,96.839996,94.080002,96.349998,45964300,95.830001
2016-02-02,95.419998,96.040001,94.279999,94.480003,37357200,93.970098
2016-02-01,96.470001,96.709999,95.400002,96.430000,40943500,95.909571
2016-01-29,94.790001,97.339996,94.349998,97.339996,64416500,96.814656
2016-01-28,93.790001,94.519997,92.389999,94.089996,55678800,93.582196
2016-01-27,96.040001,96.629997,93.339996,93.419998,133369700,92.915814
2016-01-26,99.930000,100.879997,98.070000,99.989998,75077000,99.450356
2016-01-25,101.519997,101.529999,99.209999,99.440002,51794500,98.903329
2016-01-22,98.629997,101.459999,98.370003,101.419998,65800500,100.872638


In [14]:
ts_data['2016-2']

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-02-03,95.000000,96.839996,94.080002,96.349998,45964300,95.830001
2016-02-02,95.419998,96.040001,94.279999,94.480003,37357200,93.970098
2016-02-01,96.470001,96.709999,95.400002,96.430000,40943500,95.909571


In [15]:
ts_data.ix['2016-2']

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-02-03,95.000000,96.839996,94.080002,96.349998,45964300,95.830001
2016-02-02,95.419998,96.040001,94.279999,94.480003,37357200,93.970098
2016-02-01,96.470001,96.709999,95.400002,96.430000,40943500,95.909571


**Exercise:** Filter the above dataframe to only display values from October 2015 to December 2015. 

Given that the index is in the 'wrong' order, it makes it somewhat less intuitive to work with.

We can reorder the index to make things easier.

In [16]:
ts_data.sort_index(ascending=True, inplace=True)
ts_data.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
1980-12-12,28.750000,28.875000,28.750,28.750,117258400,0.433984
1980-12-15,27.375001,27.375001,27.250,27.250,43971200,0.411342
1980-12-16,25.375000,25.375000,25.250,25.250,26432000,0.381151
1980-12-17,25.875000,25.999999,25.875,25.875,21610400,0.390586
1980-12-18,26.625000,26.750000,26.625,26.625,18362400,0.401907


**Exercise:** Filter the above dataframe to only display values after January 15 2015.


Filtering can also be done through a *truncate()*. Truncate is simply a convenience function that is equivalent to slicing. Below is an example of filtering data to just December 2015 and January 2016 observations:


In [17]:
ts_data.truncate(before='2015-12-1', after='2016-1-31')

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2015-12-01,118.750000,118.809998,116.860001,117.339996,34852400,116.706717
2015-12-02,117.339996,118.110001,116.080002,116.279999,33386600,115.652440
2015-12-03,116.550003,116.790001,114.220001,115.199997,41569500,114.578267
2015-12-04,115.290001,119.250000,115.110001,119.029999,57777000,118.387598
2015-12-07,118.980003,119.860001,117.809998,118.279999,32084200,117.641646
2015-12-08,117.519997,118.599998,116.860001,118.230003,34309500,117.591921
2015-12-09,117.639999,117.690002,115.080002,115.620003,46361400,114.996006
2015-12-10,116.040001,116.940002,115.510002,116.169998,29212700,115.543033
2015-12-11,115.190002,115.389999,112.849998,113.180000,46886200,112.569172


**Exercise:** Use the truncate function to filter the above dataframe to only display values after November 2015.

# Resampling

Resampling transforms time series data into a different frequency (e.g., converting hourly data into daily data). Pandas provide and easy way to perform these frequency conversion operations which are extremely common in  financial applications, but not limited to them only.

Resampling requires that 1) the resampling time period is specified, 2) the method to apply to the resampled data (default is mean). For those familiar with SQL, resampling is essentially a time-based **groupby** operation, followed by a reduction method on each of its groups. 

Reduction can be: 'mean','median','sum','min','max','first','last','ohlc' or other available numpy/user defined transformation.

A variety of built-in reduction time frequencies are available:

In [18]:
ts_data['2015'].resample('M', how='mean')

D:\Program Files\Anaconda\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  if __name__ == '__main__':


,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2015-01-31,111.164000,112.254000,109.332000,110.641501,65232190,108.200805
2015-02-28,125.023683,126.144738,124.031053,125.432105,59817642,123.076764
2015-03-31,126.466365,127.436363,124.985910,125.970908,51756459,123.678248
2015-04-30,127.535239,128.422380,126.224763,127.291428,47435023,124.974734
2015-05-31,128.707501,129.771001,127.663000,128.761500,47707605,126.841840
2015-06-30,128.013636,128.761364,127.240910,127.806817,39932390,126.004877
2015-07-31,125.450455,126.203637,124.340000,125.335455,48068509,123.568357
2015-08-31,113.439524,115.500000,111.290952,113.394762,76475157,112.228004
2015-09-30,113.012381,114.207142,111.523810,112.797619,57454633,111.710669


**Exercise:** Resample the date above dataframe based on the quarter end frequency on data between 1990 and 2010 using the median as the reduction method. 

**Exercise:** Describe what the output of the below means? 

Visualising the data is as simple as calling *plot()* on the required column:

In [19]:
ts_data.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
1980-12-12,28.750000,28.875000,28.750,28.750,117258400,0.433984
1980-12-15,27.375001,27.375001,27.250,27.250,43971200,0.411342
1980-12-16,25.375000,25.375000,25.250,25.250,26432000,0.381151
1980-12-17,25.875000,25.999999,25.875,25.875,21610400,0.390586
1980-12-18,26.625000,26.750000,26.625,26.625,18362400,0.401907


In [ ]:
#ts_data[['Volume']].resample('M', how='sum').plot()
plt.plot( ts_data[['Volume']].resample('M', how='sum'))

We can increase the size of the plot and render several plots at the same time:

In [ ]:
rcParams['figure.figsize'] = 15, 10
ts_data.plot(subplots=True)

**Exercise:** The period leading to the recent global financial crisis and the immediate aftermath are interesting to look at into more detail from the perspective of the adjusted closing price and the total volume of shares traded for Apple. Render separately two plots for these columns for data from 2007 to 2010.

We can use resampling to reduce the frequency of Apple share trading to annual and plot the historical variation between the min/max and the mean prices for Apple shares in each year:

In [ ]:
plt.plot( ts_data[['Adj Close']].resample('A', how='mean'))
plt.plot( ts_data[['Adj Close']].resample('A', how='min'))
plt.plot( ts_data[['Adj Close']].resample('A', how='max'))


**Exercise:** Render a graph that is the same as above, only this time use a 5 year frequency:

### Moving (rolling/running) statistics

A rolling average is a series of averages of different subsets of the full data set as defined by a filter window.

It is widely used indicator that helps smooth out price movements by filtering out the noise from random fluctuations.

In [ ]:
pd.rolling_mean(ts_data[['Adj Close']], window=5).head(10)

In [ ]:
pd.rolling_mean(ts_data[['Adj Close']], window=5).plot(style='-g')

**Exercise:** Generate rolling mean plots on the Volume column for the Apple share trading data. Determine the most 'useful' window size.

##  Shift operations

“Shifting” refers to moving data backward and forward through time. Both Series and
DataFrame have a  shift method for performing this operation.

If we wanted to calculate the difference in oil price from one year to the next (something very common in time series analysis), then pandas provides for us a method called shift(), which allows us to select a column and move the data in it up or down by a given amount. 

In our case, we want to see the difference between the values in price from one year to the next so we will shift the columns by one.

In [ ]:
ts_data.head()

In [ ]:

ts_data['shifted'] = ts_data['Adj Close'].shift(1)
ts_data

**Exercise**: Plot the positive and negative fluctuations of the oil price from year to year for the above dataset.

### Exercise: 

Read in the oil_price.csv dataset.

Convert the 'Year' feature into datetime and set it as the index.

Perform the same analysis as above using the 'shift' function and plot the InflationAdjustedPrice difference from one year to the next.

In [ ]:
%%javascript
require(['base/js/utils'],
function(utils) {
   utils.load_extensions('calico-spell-check', 'calico-document-tools', 'calico-cell-tools');
});